https://plotly.com/python/v3/3d-network-graph/
https://stackoverflow.com/questions/62601052/option-to-add-edge-colouring-in-networkx-trace-using-plotly 

In [73]:
import igraph as ig
import chart_studio.plotly as py
from plotly.offline import iplot
import plotly.graph_objs as go
import pandas as pd
import numpy as np

import networkx as nx

In [74]:
import requests
url = "https://raw.githubusercontent.com/plotly/datasets/master/miserables.json"
data = requests.get(url).json()

In [75]:
print(data.keys())
print(data['nodes'][2])

dict_keys(['nodes', 'links'])
{'name': 'Mlle.Baptistine', 'group': 1}


In [76]:
L = len(data['links'])
N = len(data['nodes'])
L, N

(254, 77)

In [77]:
links = pd.DataFrame(data["links"])
nodes = pd.DataFrame(data['nodes'])
print(links.head(3))
print(nodes.head(3))

   source  target  value
0       1       0      1
1       2       0      8
2       3       0     10
              name  group
0           Myriel      1
1         Napoleon      1
2  Mlle.Baptistine      1


In [58]:
# get number of source and target mentions per character.
nodes['sizes'] = links["source"].value_counts().sort_index().add(links["target"].value_counts().sort_index(), fill_value = 0)

# normalizing values
a, b = 6, 14
x, y = min(nodes.sizes), max(nodes.sizes)
nodes.sizes = (nodes.sizes - x) / (y - x) * (b - a) + a

In [59]:
dnd = nx.Graph()
for idx, char in nodes.iterrows():
        dnd.add_node(char.name, size = char.sizes) 

In [60]:

links["relation"] = np.random.randint(0, 2, links.shape[0])
rel_color_lookup = {0:'red', 1:'blue'}
links.relation = links.relation.apply(lambda x: rel_color_lookup[x])

In [61]:
# for each co-appearance between two characters, add an edge
for idx, link in links.iterrows():
    dnd.add_edge(link.source, link.target, weight = link.value, color = link.relation)

In [62]:
dnd.number_of_edges()

254

In [63]:
# colors = nx.get_edge_attributes(dnd,'color').values()
# weights = nx.get_edge_attributes(dnd,'weight').values()
# colors, weights

In [64]:
pos_ = nx.spring_layout(dnd, dim = 3, k = 0.8, iterations = 50) # get coordinates for nodes in spring layout
Xn, Yn, Zn = [], [], []
for k in range(dnd.number_of_nodes()): 
  Xn += [ pos_[k][0] ] 
  Yn += [ pos_[k][1] ]
  Zn += [ pos_[k][2] ]

print(len(Xn))
nodes["Xn"] = Xn
nodes["Yn"] = Yn
nodes["Zn"] = Zn

77


In [65]:
nodes

,name,group,sizes,Xn,Yn,Zn
0,Myriel,1,8.057143,0.018336,-0.177458,0.436552
1,Napoleon,1,6.000000,0.215345,-0.874937,-0.340964
2,Mlle.Baptistine,1,6.457143,-0.050715,-0.145531,0.618285
3,Mme.Magloire,1,6.457143,-0.139115,-0.185772,0.556071
4,CountessdeLo,1,6.000000,0.705701,-0.521544,0.588905
...,...,...,...,...,...,...
72,Toussaint,5,6.457143,0.537041,-0.077987,0.118683
73,Child1,10,6.228571,-0.506722,-0.219807,0.140187
74,Child2,10,6.228571,-0.335435,-0.341538,0.235886
75,Brujon,4,7.371429,0.440876,0.461547,-0.036476


In [66]:
# minmax scale groups to get only four
a, b = 0, 3
x, y = min(nodes.group), max(nodes.group)
nodes.group = np.ceil(((nodes.group - x) / (y - x) * (b - a) + a))

nodes.group.value_counts().sort_index()

0.0     3
1.0    34
2.0    22
3.0    18
Name: group, dtype: int64

In [67]:
color_lookup = {0:'aliceblue', 1:'blanchedalmond', 2:'crimson', 3:'darkblue'}

In [68]:
nodes["colors"] = nodes.group.apply(lambda x: color_lookup[int(x)])

In [69]:
nodes['description'] = ["SomeDesc"] * len(nodes)

In [70]:
# colors = nx.get_edge_attributes(G,'color').values()
# weights = nx.get_edge_attributes(G,'weight').values()

In [71]:
def trace_factory(node_cluster, name):
    return go.Scatter3d(x = node_cluster.Xn, 
                        y = node_cluster.Yn, 
                        z = node_cluster.Zn, 
                        mode = 'markers + text + lines', 
                        name = name, 
                        marker = dict(symbol = 'circle', 
                                    size = node_cluster.sizes, 
                                    color = node_cluster.colors.to_list()),
                        line = dict(color='rgb(125,125,125)', width=0.5),
                        text = node_cluster.name, 
                        visible = True,
                        hoverinfo = 'text',
                        showlegend = True,
                        customdata = np.stack([node_cluster.description, node_cluster.group], axis = 1),
                        hovertemplate = ('%{text}'+\
                        '<br><i>%{customdata[0]}</i><br>'+\
                        '<br><b>Affiliation</b>: %{customdata[1]}<br>'))

In [72]:

model_data = [trace_factory(nodes, name = "all"),
              trace_factory(nodes.query("group == 0"), name = "group0"),
              trace_factory(nodes.query("group == 1"), name = "group1"),
              trace_factory(nodes.query("group == 2"), name = "group2"),
              trace_factory(nodes.query("group == 3"), name = "group3")]


axis = dict(showbackground = False, 
            showline = False, 
            zeroline = False, 
            showgrid = False, 
            showticklabels = False, 
            showspikes = False,
            title = '')

layout = go.Layout(
    title = "Campaign Cast Network",
    width = 1000,
    height = 700,
    showlegend = True,
    scene = dict(
        xaxis = dict(axis),
        yaxis = dict(axis),
        zaxis = dict(axis)))

fig = go.Figure(data = model_data, 
                layout = layout)

iplot(fig, filename = 'Les-Miserables')